In [5]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.named_parameters="enabled" 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
%%sql
WITH
prels AS (
    SELECT
        de_partition AS annee,
        cdreseau,
        referenceprel,
        datetimeprel,
        COUNT(DISTINCT cdparametresiseeaux) AS nb_parametres,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN valtraduite
                ELSE 0
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN valtraduite
                ELSE 0
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN valtraduite
                ELSE 0
            END
        ) AS valtraduite_no3_no2
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
    GROUP BY
        annee,
        cdreseau,
        referenceprel,
        datetimeprel,
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
       int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
)

SELECT
 --   prels.cdreseau,
    prels.annee,
 --   'nitrate' AS categorie,
 --   'bilan_annuel_' || prels.annee AS periode,
    COUNT(DISTINCT prels.referenceprel) AS nb_prelevements,
    COUNT(
        DISTINCT
        CASE
            WHEN
                prels.valtraduite_no3 >= valeur_ref.limite_qualite_no3
                OR prels.valtraduite_no2 >= valeur_ref.limite_qualite_no2
                OR prels.valtraduite_no3_no2
                >= valeur_ref.limite_qualite_no3_no2
                OR prels.valtraduite_no3 / 50 + prels.valtraduite_no2 / 3
                >= valeur_ref.limite_qualite_no3_no2
                THEN prels.referenceprel
        END
    ) AS nb_dep_3critères,
    trunc((
        COUNT(
            DISTINCT
            CASE
                WHEN
                    prels.valtraduite_no3 >= valeur_ref.limite_qualite_no3
                    OR prels.valtraduite_no2 >= valeur_ref.limite_qualite_no2
                    OR prels.valtraduite_no3_no2
                    >= valeur_ref.limite_qualite_no3_no2
                    OR prels.valtraduite_no3 / 50 + prels.valtraduite_no2 / 3
                    >= valeur_ref.limite_qualite_no3_no2
                    THEN prels.referenceprel
            END
        )::float
        /
        COUNT(DISTINCT prels.referenceprel)::float
    )*100) AS percent_3critères,

    COUNT(DISTINCT CASE WHEN valtraduite_no2 IS NOT NULL THEN  prels.referenceprel END) AS nb_prelevements_n02,
    COUNT(
        DISTINCT
        CASE
            WHEN
               prels.valtraduite_no2 >= valeur_ref.limite_qualite_no2
                THEN prels.referenceprel
        END
    ) AS nb_dep_n02,
    trunc((
        COUNT(
            DISTINCT
            CASE
                WHEN
                    prels.valtraduite_no2 >= valeur_ref.limite_qualite_no2
                    THEN prels.referenceprel
            END
        )::float
        /
       COUNT(DISTINCT CASE WHEN valtraduite_no2 IS NOT NULL THEN  prels.referenceprel END)::float
    )*100) AS percent_n02,

    COUNT(DISTINCT CASE WHEN valtraduite_no3 IS NOT NULL THEN  prels.referenceprel END) AS nb_prelevements_n03,
    COUNT(
        DISTINCT
        CASE
            WHEN
               prels.valtraduite_no3 >= valeur_ref.limite_qualite_no3
                THEN prels.referenceprel
        END
    ) AS nb_dep_n03,
    trunc((
        COUNT(
            DISTINCT
            CASE
                WHEN
                    prels.valtraduite_no3 >= valeur_ref.limite_qualite_no3
                    THEN prels.referenceprel
            END
        )::float
        /
       COUNT(DISTINCT CASE WHEN valtraduite_no2 IS NOT NULL THEN  prels.referenceprel END)::float
    )*100) AS percent_n03

FROM prels
CROSS JOIN valeur_ref
GROUP BY /*prels.cdreseau,*/ prels.annee


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,nb_prelevements,nb_dep_3critères,percent_3critères,nb_prelevements_n02,nb_dep_n02,percent_n02,nb_prelevements_n03,nb_dep_n03,percent_n03
2020,129386,1745,1.0,129385,27,0.0,129381,1703,1.0
2025,18138,285,1.0,18138,1,0.0,17681,283,1.0
2021,126493,1823,1.0,126487,17,0.0,126490,1802,1.0
2022,128745,1605,1.0,128736,16,0.0,128737,1586,1.0
2024,127734,1980,1.0,127733,16,0.0,127733,1960,1.0
2023,129172,1447,1.0,129171,35,0.0,129154,1405,1.0
